# Imports

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Constants

In [2]:
torch.manual_seed(1337)
training_style = "scale"
if training_style == "scale":
    block_size = 32
    batch_size = 512
    split_pct = 0.9
    max_iters = 5000
    eval_interval = 500
    learning_rate = 3e-4 #1e-3
    eval_iters = 200
    n_embd = 384 #64
    n_layer = 6 # Number of block layers
    n_head = 6
    dropout = 0.2
else:
    block_size = 32
    batch_size = 256
    split_pct = 0.9
    max_iters = 500
    eval_interval = 100
    learning_rate = 1e-3
    eval_iters = 200
    n_embd = 64
    n_layer = 1 # Number of block layers
    n_head = 1
    dropout = 0.2

In [3]:
# Set GPU device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(device)

cuda


# Dataloader

In [4]:
chars = ['\\', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', '-', '.', '=', 'e']
vocab_size = len(chars)
print("Number of tokens: {}".format(vocab_size))

Number of tokens: 16


In [5]:
encoder = { ch:i for i,ch in enumerate(chars) }
decoder = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [encoder[c] if c in chars else print (s) for c in s]
decode = lambda l: ''.join([decoder[i] for i in l])

In [6]:
def get_batch(bs):
    numbers = torch.randint(0, 1000000, (bs, 2))
    x = []
    y = []
    for ix in range(bs):
        # Create equation with solution digits reversed
        equation = "{}+{}={}".format(numbers[ix,0], numbers[ix,1], str((numbers[ix,0] + numbers[ix,1]).item())[::-1])
        # Pick a random place in equation after = and try to predict the next character
        string_idx = torch.randint(equation.index("=")+1, len(equation), (1,))
        # Add padding so input and target both have lengths equal to block size
        x.append(torch.tensor(encode("\\" * (block_size - string_idx) + equation[:string_idx])))
        y.append(torch.tensor(encode("\\" * (block_size - string_idx - 1) + equation[:string_idx+1])))
    return torch.stack(x).to(device), torch.stack(y).to(device)

In [7]:
x, y = get_batch(256)
x.size(), y.size()

(torch.Size([256, 32]), torch.Size([256, 32]))

In [8]:
idx = 0
decode(x[idx].tolist()), decode(y[idx].tolist())

('\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\387415+456957=273',
 '\\\\\\\\\\\\\\\\\\\\\\\\\\\\387415+456957=2734')

# Model

In [9]:
# Obtains average loss over multiple batches
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, T = get_batch(batch_size)
            logits, loss = m(X, T)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [10]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=False) # (C, 3C)
        self.register_buffer('tril', torch.tril(torch.ones(1, 1, block_size, block_size)))
        self.dropout1 = nn.Dropout(dropout)
        self.proj = nn.Linear(n_embd, n_embd) # (C, C)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x):
        # x has shape (B,T,C) - because of initial embeddings
        B, T, C = x.shape

        q, k, v = self.c_attn(x).split(n_embd, dim=2) # (B, T, C) @ (C, 3C) --> (B, T, 3C) splits -> (B, T, C)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1,2) # (B, nh, T, hs)

        # affinities
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, nh, T, hs) @ (B, nh, hs, T) ---> (B,nh,T,T)
        wei = wei.masked_fill(self.tril[:, :, :T, :T] == 0, float('-inf')) # (B, nh, T, T)
        wei = F.softmax(wei, dim=-1) # (B, nh, T, T)
        wei = self.dropout1(wei) # (B, nh, T, T)
        out = wei @ v # (B, nh, T, T) @ (B, nh, T, hs) --> (B, nh, T, hs)
        out = out.transpose(1, 2).contiguous().view(B,T,C) # (B, T, nh, hs) -> (B, T, C)
        out = self.dropout2(self.proj(out)) # (B, T, C) @ (C, C) --> (B, T, C)
        return out

In [11]:
class FeedForward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # Projection layer
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [12]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.sa = MultiHeadAttention(n_head)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [13]:
class GptModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tkn_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tkn_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, ignore_index=encode("\\")[0])
            
        return logits, loss

    def generate(self, no_sums):
        # idx is (B, T) array of indices in the current context
        x, y = get_batch(no_sums)
        max_new_tokens = 12
        results = []
        for idx, eqn in enumerate(x):
            for _ in range(max_new_tokens):
                # crop idx to last block_size tokens
                eqn_cond = eqn[:, -block_size:]
                # get predictions
                logits, loss = self(eqn_cond)
                # focus only on last time step
                logits = logits[:, -1, :]
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # )B, C)
                # sample from distribution
                eqn_next = torch.multinomial(probs, num_samples=1) # (B, 1)
                # append sampled index to the running sequence
                eqn = torch.cat((eqn, eqn_next), dim=1) # (B, T+1)
            results.append((decode(eqn[0].tolist())), decode(y[idx].tolist()))
        return results

In [14]:
m = GptModel()
m.to(device)

GptModel(
  (token_embedding_table): Embedding(16, 384)
  (position_embedding_table): Embedding(32, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (dropout1): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (sa): MultiHeadAttention(
        (c_attn): Linear(in_features=384, out_features=1152, bias=False)
        (dropout1): Dropout(p=0.2, 

In [15]:
# Number of parameters in model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

7.118608 M parameters


# Training

In [16]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [17]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch(batch_size)
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.9586, val loss 2.9609
step 500: train loss 2.0576, val loss 2.0578
step 1000: train loss 2.0519, val loss 2.0524
step 1500: train loss 2.0491, val loss 2.0499
step 2000: train loss 2.0476, val loss 2.0474
step 2500: train loss 2.0466, val loss 2.0465
step 3000: train loss 2.0405, val loss 2.0402
step 3500: train loss 1.9344, val loss 1.9346
step 4000: train loss 1.9244, val loss 1.9246
step 4500: train loss 1.8970, val loss 1.8962
step 4999: train loss 1.8902, val loss 1.8890


# Inference

In [18]:
def gen_sums(num):
    numbers = torch.randint(0, 1000000, (num, 2))
    x = []
    y = []
    for ix in range(num):
        # Create equation with solution digits reversed
        eqn = "{}+{}=".format(numbers[ix,0], numbers[ix,1], str((numbers[ix,0] + numbers[ix,1]).item()))
        answ = (numbers[ix,0] + numbers[ix,1]).item()
        # Add padding so input and target both have lengths equal to block size
        x.append(torch.tensor(encode("\\" * (block_size - len(eqn)) + eqn)))
        y.append(answ)
    return torch.stack(x).to(device), y

In [19]:
def generate(no_sums):
    # idx is (B, T) array of indices in the current context
    x, answer = gen_sums(no_sums)
    max_new_tokens = 7
    results = []
    for idx, eqn in enumerate(x):
        eqn = eqn.unsqueeze(0)
        for _ in range(max_new_tokens):
            # crop idx to last block_size tokens
            eqn_cond = eqn[:,-block_size:]
            # get predictions
            logits, loss = m(eqn_cond)
            # focus only on last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # )B, C)
            # sample from distribution
            eqn_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            eqn = torch.cat((eqn, eqn_next), dim=1) # (B, T+1)
        results.append((decode(eqn[0].tolist()).replace("\\",""), answer[idx]))
    return results

In [20]:
inference = generate(no_sums=10)
for calc in inference:
    print("Prediction: {}\nReal Result:{}".format(calc[0], str(calc[1])[::-1]))

Prediction: 830388+747849=7422351
Real Result:7328751
Prediction: 610217+289100=7832781
Real Result:713998
Prediction: 274443+152814=7886341
Real Result:752724
Prediction: 914668+858050=8950771
Real Result:8172771
Prediction: 64427+72591=0907847
Real Result:810731
Prediction: 553931+800555=6621531
Real Result:6844531
Prediction: 590516+429465=1418101
Real Result:1899101
Prediction: 139994+420297=1586051
Real Result:192065
Prediction: 787588+840318=6578371
Real Result:6097261
Prediction: 192213+673673=6806381
Real Result:688568


# Save the model

In [21]:
#import time
#torch.save(m.state_dict(), './models/addition_calculator_{}.pt'.format(time.strftime("%Y-%m-%d-%H:%M")))